In [1]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('https://online.stat.psu.edu/onlinecourses/sites/stat501/files/data/iqsize.txt',sep = '\t')
Xdf=df[['Brain','Height']]
ydf = df[['PIQ']]
formula = 'PIQ~Brain + Height'
df.head()

,PIQ,Brain,Height,Weight
0,124,81.69,64.5,118
1,150,103.84,73.3,143
2,128,96.54,68.8,172
3,134,95.15,65.0,147
4,110,92.88,69.0,146


In [3]:
y=ydf.values
X=Xdf.values
Xo=sm.add_constant(X)
Xdfo=sm.add_constant(Xdf)
n,k = X.shape
sl = .95
alpha = 1 - sl

In [4]:
model = ols(formula, data=df).fit()

$$\hat{{\beta}} = ({X}^\prime {X})^{-1} {X}^\prime {y}$$

$$\hat{y} = {x}^*\hat{{\beta}}={x}^*({X}'{X})^{-1}{X}'{y}$$

$$\hat{\sigma}_e^2 = \frac{1}{n-k-1}({y} - {X}\hat{{\beta}})'
  ({y} - {X}\hat{{\beta}})$$


  $$\hat{y} \pm t_{\text{mult}} \hat{\sigma}_e \sqrt{1 + {x}^* ({X}^\prime {X})^{-1} ({x}^*)'}$$


In [5]:
Xstar = {**dict(zip(Xdfo.columns, Xdfo.iloc[0, :])), "Brain": 90, "Height": 70}
xstar = np.asarray([list(Xstar.values())])
Xstar,xstar

({'const': 1.0, 'Brain': 90, 'Height': 70}, array([[ 1., 90., 70.]]))

In [6]:
model.get_prediction(Xstar).summary_frame(alpha=alpha).obs_ci_lower[0], model.get_prediction(Xstar).summary_frame(alpha=0.05).obs_ci_upper[0]

(65.34688254130864, 145.9313730474744)

In [7]:
from scipy.stats import t
from numpy.linalg import inv
Bhat = inv(Xo.T@Xo)@Xo.T@y
MSE = (1/(n-k-1)*(y-Xo@Bhat).T@(y-Xo@Bhat))[0,0]
tmult = t.ppf(1-alpha/2, df=(n-k-1))
ME = tmult*np.sqrt(MSE*(1+(xstar@np.linalg.inv(Xo.T@Xo)@xstar.T)))[0,0]
yhat = (xstar@Bhat)[0,0]
yhat-ME, yhat+ME

(65.34688254131356, 145.9313730474791)

In [8]:
tmult = model.get_prediction(Xstar).dist.ppf(1-alpha/2,*model.get_prediction(Xstar).dist_args)
ME = tmult*np.sqrt(model.mse_resid*(1+xstar@model.normalized_cov_params.values@xstar.T))[0,0]
yhat = model.predict(Xstar)[0]
yhat-ME, yhat+ME

(65.34688254130862, 145.93137304747444)